In [165]:
# import the libraries
import time
import re
import random as rd
import pandas as pd
import numpy as np
import collections
from tqdm import tqdm
import pickle
import datetime
from dateutil.relativedelta import relativedelta

import requests
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager

from newspaper import Article

In [166]:
# Driver setting
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3"
headers = {
    "User-Agent": user_agent
}

def setWebdriver():
    service = ChromeService(executable_path=ChromeDriverManager().install())    # 크롬 드라이버 최신 버전 설정

    options = ChromeOptions()
    options.add_argument('user-agent=' + user_agent)
    options.add_argument('--start-maximized') #브라우저가 최대화된 상태로 실행됩니다.
    # options.add_argument('headless') #headless모드 브라우저가 뜨지 않고 실행됩니다.
    #options.add_argument('--window-size= x, y') #실행되는 브라우저 크기를 지정할 수 있습니다.
    #options.add_argument('--start-fullscreen') #브라우저가 풀스크린 모드(F11)로 실행됩니다.
    #options.add_argument('--blink-settings=imagesEnabled=false') #브라우저에서 이미지 로딩을 하지 않습니다.
    options.add_argument('--mute-audio') #브라우저에 음소거 옵션을 적용합니다.
    options.add_argument('incognito') #시크릿 모드의 브라우저가 실행됩니다.
    driver = webdriver.Chrome(service=service, options=options)

    return driver

In [171]:
# 원하는 횟수만큼 Scroll down

def scrollDown(driver, num: int): 
    
    for _ in tqdm(range(0, num), desc='Scrolling Down...', leave='False'):
        driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(rd.uniform(0.1, 0.3))    

    return

In [169]:
# start_date부터 end_date까지 3개월 단위로 나누기

def getPeriods(start_date, end_date):
    
    periods = []
    while (start_date < end_date):
        if (start_date + relativedelta(months=3) > end_date):
            periods.append([start_date.strftime("%Y.%m.%d"), end_date.strftime("%Y.%m.%d")])

        else:
            periods.append([start_date.strftime("%Y.%m.%d"), (start_date + relativedelta(months=3)).strftime("%Y.%m.%d")])
        start_date += relativedelta(months=3)
    # print(periods)

    return periods

In [176]:
# 언론사 세팅

def setCompany():

    company_names = ["YTN", "JTBC", "MBC", "SBS", "국민일보", "매일경제", "조선일보", "중앙일보", "아시아경제", "한국경제", "KBS", "한겨례", "경향신문"]
    company_codes = {
        "YTN": "1052",
        "JTBC": "1437",
        "MBC": "1214",
        "SBS": "1055",
        "국민일보": "1005",
        "매일경제": "1009",
        "조선일보": "1023",
        "중앙일보": "1025",
        "아시아경제": "1277",
        "한국경제": "1015",
        "KBS": "1056",
        "한겨례": "1028",
        "경향신문": "1032",
    }

    return company_names, company_codes

In [173]:
# 기간 세팅 (3개월 단위로 나누기)

start_date = datetime.datetime.strptime("2020.09.01", "%Y.%m.%d")
end_date = datetime.datetime.strptime("2023.12.31", "%Y.%m.%d")
periods = getPeriods(start_date, end_date)

periods[0][0] = "2020.09.02"
print(periods)

[['2020.09.02', '2020.12.01'], ['2020.12.01', '2021.03.01'], ['2021.03.01', '2021.06.01'], ['2021.06.01', '2021.09.01'], ['2021.09.01', '2021.12.01'], ['2021.12.01', '2022.03.01'], ['2022.03.01', '2022.06.01'], ['2022.06.01', '2022.09.01'], ['2022.09.01', '2022.12.01'], ['2022.12.01', '2023.03.01'], ['2023.03.01', '2023.06.01'], ['2023.06.01', '2023.09.01'], ['2023.09.01', '2023.12.01'], ['2023.12.01', '2023.12.31']]


In [197]:
driver = setWebdriver()

In [136]:
company_code = companies[companies_name[0]]
query = querys[0]
start_date = periods[0][0]    # 시작 날짜
end_date = periods[0][1]    # 끝 날짜

In [137]:
url = f'https://search.naver.com/search.naver?where=news&query={query}&sm=tab_clk.jou&sort=0&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked={company_code}&nso=so%3Ar%2Cp%3Afrom20210520to20240520&is_sug_officeid=0&office_category=0&service_area=0'
print(url)

https://search.naver.com/search.naver?where=news&query=국민의힘&sm=tab_clk.jou&sort=0&photo=0&field=0&pd=3&ds=2020.09.02&de=2020.12.01&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked=1052&nso=so%3Ar%2Cp%3Afrom20210520to20240520&is_sug_officeid=0&office_category=0&service_area=0


In [198]:
driver.get(url)

In [199]:
# 아래로 스크롤 할 수 있을 만큼 스크롤 다운
scrollDown(driver, 99)

Scrolling Down...: 100%|██████████| 99/99 [00:21<00:00,  4.66it/s]


In [200]:
news_list = driver.find_elements(By.CLASS_NAME, "news_tit")

In [201]:
print(len(news_list))

1000


In [203]:
news_url = news_list[0].get_attribute('href')
print(news_url)

https://www.ytn.co.kr/_ln/0101_202011301050011782


In [204]:
article = Article(news_url, language="ko")

In [ ]:
current_url = news_url
article = Article(current_url, language="ko")
article.download()
article.parse()

date = article.publish_date;    print(f'date: {date}')
authors = article.authors;      print(f'authors: {authors}')
title = article.title;  print(f'title: {title}')
text = article.text;    print(f'text: {text}')
print(f'{"-"*200}')

# dates.append(date)
# titles.append(title)
# texts.append(text)

In [191]:
def main():
    
    # 언론사 세팅
    company_names, comapny_codes = setCompany()

    # 기간 세팅 (3개월 단위로 나누기)
    start_date = datetime.datetime.strptime("2020.09.01", "%Y.%m.%d")
    end_date = datetime.datetime.strptime("2023.12.31", "%Y.%m.%d")
    periods = getPeriods(start_date, end_date)

    # 검색어 세팅
    querys = ["국민의힘", "더불어민주당"]
    
    
    driver = setWebdriver()

    for comany in tqdm(company_names, desc="Companies..."):
        company_code = comapny_codes[comany]

        for period in periods:
            start_date = period[0]
            end_date = period[1]

            for query in querys:
                url = f'https://search.naver.com/search.naver?where=news&query={query}&sm=tab_clk.jou&sort=0&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&docid=&related=0&mynews=1&office_type=1&office_section_code=1&news_office_checked={company_code}&nso=so%3Ar%2Cp%3Afrom20210520to20240520&is_sug_officeid=0&office_category=0&service_area=0'

                driver.get(url)
                time.sleep(1)

                scrollDown(driver, 99)  # 아래로 스크롤 할 수 있을 만큼 스크롤 다운

                news_list = driver.find_elements(By.CLASS_NAME, "news_tit")

                for news in news_list:
                    news_url = news.get_attribute('herf')

                    driver.execute_script(f"window.open('{news_url}');")     # 새 탭 열기
                    driver.switch_to.window(driver.window_handles[1])       # 새 탭으로 이동

                    article = Article(current_url, language="ko")
                    article.download()
                    article.parse()

                    date = article.publish_date;    print(f'date: {date}')
                    authors = article.authors;      print(f'authors: {authors}')
                    title = article.title;  print(f'title: {title}')
                    text = article.text;    print(f'text: {text}')
                    print(f'{"-"*200}')
                    
                    time.sleep(1)
                    driver.close()                                          # 현재 탭 닫기
                    driver.switch_to.window(driver.window_handles[0])       # 0번째 탭으로 이동

In [ ]:
if __name__ == "__main__":
    main()

In [195]:
# 첫 번째 탭으로 이동
driver.switch_to.window(driver.window_handles[0])

news_list = driver.find_elements(By.CLASS_NAME, 'news_tit')
print(len(news_list))

0


In [206]:
df = pd.DataFrame(columns=["Date", "Title", "Text"])
df

,Date,Title,Text


In [27]:
df = pd.read_csv('../data/JTBC.csv')
df

,Unnamed: 0,Query,Date,Title,Text
0,0,국민의힘,2020-11-28 19:31:00+09:00,"김종인 ""추미애 역겹다""…민주 ""국민의힘 막말 더 역겨워""",[앵커]여야의 대리전은 오늘(28일)도 계속됐습니다. 서로 말폭탄까지 주고받았는데요...
1,1,국민의힘,2020-11-27 14:11:00+09:00,"추미애 ""판사 사찰, 국민께 보고할 의무…법·절차 따라 진행""","[앵커]현재 법무부와 대검찰청 상황 어떤지 연결해보겠습니다.이상엽 기자, 추미애 법..."
2,2,국민의힘,2020-11-24 20:25:00+09:00,"민주당 ""윤석열이 자초""…국민의힘 ""추미애 무법전횡""","[앵커]이번에는 정치권 반응을 확인해보겠습니다.박소연 기자, 청와대 입장 나왔습니까..."
3,3,국민의힘,2020-11-20 20:52:00+09:00,국민의힘 PK 의원들 '가덕도 특별법' 발의…내분 격화,[앵커]부산 가덕도 신공항을 놓고 국민의힘 안에서 갈등이 커지고 있습니다. 부산지역...
4,4,국민의힘,2020-11-26 09:22:00+09:00,"[인터뷰] 정의당 ""문 대통령, 추미애 손 들어준 것…국민 위한 메시지 있어야""",■ 인용보도 시 프로그램명 'JTBC 아침&'을 밝혀주시기 바랍니다. 저작권은 JT...
...,...,...,...,...,...
20708,20708,더불어민주당,2023-12-08 16:37:00+09:00,"강정애 보훈장관 후보자, 50년 만기 주담대 막차 탔다",강정애 국가보훈부 장관 후보자가 올해 시중은행에서 50년 만기 주택담보대출을 받은 ...
20709,20709,더불어민주당,2023-12-06 15:30:00+09:00,박용진 “변희재와 연대 난데없다…송영길 신당 잘 안굴러갈 것”,박용진 더불어민주당 의원이 오늘(6일) JTBC 유튜브 라이브 '장르만 여의도'에 ...
20710,20710,더불어민주당,2023-12-05 15:56:00+09:00,"""40대와 14살의 사랑?"" 과거 판결 직격…조희대 후보자 대답은 [현장영상]",오늘(5일) 국회에서 열린 대법원장 후보자 인사청문회에서 전용기 더불어민주당 의원은...
20711,20711,더불어민주당,2023-12-05 20:14:00+09:00,"조희대 대법원장 후보자 ""재판지연 실타래 풀겠다""…야당도 긍정?","[앵커]사법부를 이끄는 대법원장 자리, 두달 넘게 비어있었는데요. 오늘(5일)부터 ..."


In [28]:
df.isnull().sum()

Unnamed: 0     0
Query          0
Date          14
Title         14
Text          20
dtype: int64

In [5]:
import pandas as pd
tmp1 = pd.DataFrame(columns=["Query", "Date", "Title", "Text"])

In [18]:

tmp3 = pd.DataFrame({"Query": "123", "Date": "12", "Title": '123', "Text": '123'}, index=([len(tmp)]))
tmp = pd.concat([tmp, tmp3])

In [19]:
tmp

,Query,Date,Title,Text
0,123,12,123,123
0,123,12,123,123
2,123,12,123,123
3,123,12,123,123
